In [2]:
import pandas as pd
from sktime.datasets import load_tsf_to_dataframe

In [8]:
df, metadata = load_tsf_to_dataframe('../../raw_data/australian_electricity_demand_dataset.tsf')
#df.drop(columns=['series_name', 'state'], inplace=True)
df.head()

series_value
series_name state timestamp                        
T1          NSW   2002-01-01 00:00:00   5714.045004
                  2002-01-01 00:30:00   5360.189078
                  2002-01-01 01:00:00   5014.835118
                  2002-01-01 01:30:00   4602.755516
                  2002-01-01 02:00:00   4285.179828

In [4]:
df.columns

Index(['series_value'], dtype='object')